<a href="https://colab.research.google.com/github/coen2812023/semantic_search/blob/main/FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import requests
import os

data_url = "https://raw.githubusercontent.com/jamescalam/data/main/sentence_embeddings_15K/"
s=[]
# create data directory to store data
if not os.path.exists('./data'):
    os.mkdir('./data')

# download the numpy binary files (dense vectors)
for i in range(57):
    if i < 10:
        i = '0' + str(i)
    res = requests.get(data_url+f"embeddings_{i}.npy")

    with open(f'./data/embeddings_{i}.npy', 'wb') as fp:
        for chunk in res:
            fp.write(chunk)
           
    print('.', end='')

# and download the respective text file
res = requests.get(f"{data_url}sentences.txt")
with open(f"./data/sentences.txt", 'wb') as fp:
    for chunk in res:
        fp.write(chunk)

.........................................................

In [ ]:
# urls = [
#     'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
#     'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
#     'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
#     'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
#     'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
#     'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
#     'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
# ]

In [ ]:
import pandas as pd

import requests
from io import StringIO
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [ ]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [ ]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

4802

In [ ]:


# # each of these dataset have the same structure, so we loop through each creating our sentences data
# for url in urls:
#     res = requests.get(url)
#     # extract to dataframe
    
#     smalldata = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
#     # data=data.append(smalldata,ignore_index=False)
#     # add to columns 1 and 2 to sentences list
#     sentences.extend(smalldata[1].tolist())
#     sentences.extend(smalldata[2].tolist())

In [ ]:
# smalldata.head()

In [ ]:
# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [ ]:
len(sentences)

4802

In [ ]:
import numpy as np
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 64.6 MB/s eta 0:00:00


In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=b68ff3d04b668c72e2eefa922c19f25843bb5dca5913ff7a3aa32ad02dd2365b
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
!git clone https://github.com/jamescalam/data.git data-embeddings

Cloning into 'data-embeddings'...
remote: Enumerating objects: 134, done.
remote: Total 134 (delta 0), reused 0 (delta 0), pack-reused 134
Receiving objects: 100% (134/134), 79.25 MiB | 31.51 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
# path = '/content/data-embeddings/sentence_embeddings_15K/'
# sentence_embeddings = []
# for i in range(0,57):
#     # if i < 10:
#     #     i = '0' + str(i)
#     res = path+f"embeddings_{i}.npy"
#     print(res)
#     sm = np.load(res)
#     sentence_embeddings.append(sm)

In [ ]:
# arr = np.concatenate(sentence_embeddings)

In [ ]:
# arr.shape

(14504, 768)

In [ ]:
sentences = sentences[:100]

In [ ]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(100, 768)

In [ ]:
# with open(f'./sim_sentences/embeddings_X.npy', 'wb') as fp:
#     np.save(fp, sentence_embeddings[0:256])

In [ ]:
# # saving data
# split = 256
# file_count = 0
# for i in range(0, sentence_embeddings.shape[0], split):
#     end = i + split
#     if end > sentence_embeddings.shape[0] + 1:
#         end = sentence_embeddings.shape[0] + 1
#     file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
#     with open(f'./sim_sentences/embeddings_{file_count}.npy', 'wb') as fp:
#         np.save(fp, sentence_embeddings[i:end, :])
#     print(f"embeddings_{file_count}.npy | {i} -> {end}")
#     file_count = int(file_count) + 1

In [ ]:
import faiss
# sentence_embeddings = arr

In [ ]:
d = sentence_embeddings.shape[1]
d

768

In [ ]:
index = faiss.IndexFlatL2(d)

In [ ]:
index.is_trained

True

In [ ]:
index.add(sentence_embeddings)

In [ ]:
index.ntotal

100

In [ ]:
sentences

In [ ]:
k = 4
xq = model.encode(["A young boy is jumping into water"])

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[44  6 42 46]]
CPU times: user 3.89 ms, sys: 0 ns, total: 3.89 ms
Wall time: 15 ms


In [ ]:
[f'{i}: {sentences[i]}' for i in I[0]]

['44: A young boy is jumping into water',
 '6: A surfer is riding the wave',
 '42: A baby is splashing inside a blue paddling pool',
 '46: A young boy is wearing a blue patterned swim suit, a black and yellow swim cap and has blue swim goggles on her head']

In [ ]:
nlist = 10
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [ ]:
index.is_trained

False

In [ ]:
index.train(sentence_embeddings)
index.is_trained

True

In [ ]:
index.add(sentence_embeddings)
index.ntotal

100

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[44  6 42 46]]
CPU times: user 2.23 ms, sys: 987 µs, total: 3.22 ms
Wall time: 7.33 ms


In [ ]:
index.nprobe = 10

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[44  6 42 46]]
CPU times: user 3.12 ms, sys: 2 µs, total: 3.13 ms
Wall time: 13.3 ms


In [ ]:
%%time
D, I = index.search(xq, k)
print(I)

[[44  6 42 46]]
CPU times: user 2.27 ms, sys: 1.04 ms, total: 3.31 ms
Wall time: 10.1 ms


In [ ]:
[f'{i}: {sentences[i]}' for i in I[0]]

['44: A young boy is jumping into water',
 '6: A surfer is riding the wave',
 '42: A baby is splashing inside a blue paddling pool',
 '46: A young boy is wearing a blue patterned swim suit, a black and yellow swim cap and has blue swim goggles on her head']